<a href="https://colab.research.google.com/github/hamzaodeh/Group-A-DLI-Assignment/blob/main/CNN(Hamza).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [159]:
import torch
print(torch.__version__)

2.8.0+cu126


In [160]:
import os
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim

from opacus import PrivacyEngine
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [161]:
import pandas as pd
import numpy as np


from typing                       import Tuple, List
from sklearn.preprocessing import StandardScaler
#from sklearn.preprocessing        import RobustScaler
from sklearn.utils import resample
from torch.utils.data             import DataLoader
from opacus.utils.uniform_sampler import UniformWithReplacementSampler


class DatasetLoader:

    def __init__(self, df: pd.DataFrame, num_clients:int, sample_rate:float, weights:List[float]=None) -> None:

        self._num_clients  = num_clients
        (self._X_train, self._y_train, self._X_test, self._y_test) = self._load_train_test_data(df)
        #self._generator    = prng.create_random_device_generator("/dev/urandom")
        self._sample_rate  = sample_rate
        self._clients_data = self._assign_data_to_clients(self._X_train, self._y_train, weights)


    @property
    def clients_data(self) -> dict:
        return self._clients_data

    def get_train_data(self) -> DataLoader:
        train_dataset = [(torch.tensor(self._X_train[i], dtype=torch.float32), torch.tensor(self._y_train[i], dtype=torch.float32)) for i in range(len(self._X_train))]
        return DataLoader(train_dataset, batch_size=64, num_workers=1, pin_memory=False)


    def get_test_data(self) -> DataLoader:
        test_dataset = [(torch.tensor(self._X_test[i], dtype=torch.float32), torch.tensor(self._y_test[i], dtype=torch.float32)) for i in range(len(self._X_test))]
        return DataLoader(test_dataset, batch_size=64, num_workers=1, pin_memory=False)

    def _load_train_test_data(self, df:pd.DataFrame) -> Tuple[np.ndarray, np.array, np.ndarray, np.array]:

        '''
        count_class_0, count_class_1 = df.Label.value_counts()

        df_class_0 = df[df['Label'] == 0]
        df_class_1 = df[df['Label'] == 1]

        df_class_1_over = df_class_1.sample(count_class_0, replace=True)
        df = pd.concat([df_class_0, df_class_1_over], axis=0)
        '''

        attack_indices     = df.loc[df.Label == 1].index
        normal_indices = df.loc[df.Label == 0].index


        attack_instances=np.random.random(len(attack_indices))<0.80
        normal_instances=np.random.random(len(normal_indices))<0.80



        test_attack = df.iloc[attack_indices[~attack_instances]]
        test_normal = df.iloc[normal_indices[~normal_instances]]
        test_df = pd.concat([test_attack, test_normal])


        train_attack = df.iloc[attack_indices[attack_instances]]
        train_normal = df.iloc[normal_indices[normal_instances]]
        train_df = pd.concat([train_attack, train_normal])

        train_df = train_df.sample(frac=1).reset_index(drop=True)
        test_df  = test_df.sample(frac=1).reset_index(drop=True)


        attack = train_df.loc[train_df.Label == 1]
        normal = train_df.loc[train_df.Label == 0][:len(attack)]
        train_df_even = pd.concat([attack, normal])
        train_df_even = train_df_even.sample(frac=1).reset_index(drop=True)


        X_train = train_df_even.drop('Label', axis=1).values
        y_train = train_df_even.Label.values

        X_test = test_df.drop('Label', axis=1).values
        y_test = test_df.Label.values

        scaling = StandardScaler()
        X_train = scaling.fit_transform(X_train)
        X_test = scaling.transform(X_test)

        return X_train, y_train, X_test, y_test

    def _assign_data_to_clients(self, X_train:np.ndarray, y_train:np.ndarray, weights:List[float]) -> dict:

            num_samples_per_client = X_train.shape[0] // self._num_clients
            clients_all_data = {}

            for i in range(self._num_clients):
                batched_x = self._X_train[i*num_samples_per_client:(i+1)*num_samples_per_client]
                batched_y = self._y_train[i*num_samples_per_client:(i+1)*num_samples_per_client]

                client_dataset = [(batched_x[i], batched_y[i]) for i in range(len(batched_x))]
                data = DataLoader(client_dataset, batch_size=64,
                                  num_workers=1, pin_memory=False)

                clients_all_data[f'client_{i}'] = data
        #else:
            # TODO
            #clients_all_data = {}
            return clients_all_data

In [162]:
MAX_GRAD_NORM = 1.
EPSILON = 20.0
#DELTA = 1e-7
DELTA = 1e-4
EPOCHS = 6
#NOISE= 1.3
NOISE= 0.5

In [163]:
class Server:
    def __init__(self, model):
        # Accept a model directly (CNN in this case)
        self._model = model
        self._client_parameters = []
        self._client_samples_processed = []

    @property
    def parameters(self):
        return self._model.parameters()

    @property
    def model(self):
        return self._model

    def add_client_parameters(self, parameters, num_samples):
        self._client_parameters.append(parameters)
        self._client_samples_processed.append(num_samples)

    def _weight_scaling_factor(self, i:int) -> float:
        global_count = sum(self._client_samples_processed)
        local_count  = self._client_samples_processed[i]
        return local_count / global_count

    def _scale_model_parameters(self, i:int) -> None:
        scalar     = self._weight_scaling_factor(i)
        parameters = list(self._client_parameters[i])
        new_parameters = [parameters[i]*scalar for i in range(len(parameters))]
        with torch.no_grad():
            for idx, (param, new_param) in enumerate(zip(self._client_parameters[i], new_parameters)):
                self._client_parameters[i][idx] = new_param

    def aggregate_parameters(self) -> None:
        for i in range(len(self._client_parameters)):
            self._scale_model_parameters(i)

        aggregated_parameters = [torch.zeros(list(self._client_parameters[0])[j].shape, device=device)
                                for j in range(len(list(self._client_parameters[0])))]

        for j in range(len(list(self._client_parameters[0]))):
            for i in range(len(self._client_parameters)):
                aggregated_parameters[j] += self._client_parameters[i][j]

        with torch.no_grad():
            for model_param, new_param in zip(self._model.parameters(), aggregated_parameters):
                model_param.data = new_param.view_as(model_param.data)

        self._client_parameters        = []
        self._client_samples_processed = []


class Client:
    def __init__(self, train_loader, noise_multiplier, max_grad_norm, DELTA,
                 input_features=36, num_classes=2):
        self._model      = CNN(input_features=input_features, num_classes=num_classes).to(device)
        self._lr         = args.lr
        self._loss       = torch.nn.CrossEntropyLoss()

        self.train_loader= train_loader
        self.privacy_engine = PrivacyEngine(secure_mode=False)
        self._optimizer  = torch.optim.SGD(self._model.parameters(), self._lr)

        # Make model DP
        self._model, self._optimizer, self.train_loader = self.privacy_engine.make_private(
            module=self._model,
            optimizer= self._optimizer,
            data_loader=self.train_loader,
            noise_multiplier=noise_multiplier,
            max_grad_norm=max_grad_norm,
        )

    @property
    def model(self):
        return self._model

    @property
    def parameters(self):
        return self._model.parameters()

    def train_one_epoch(self, train_loader: DataLoader, device:torch.device, epoch:int, client:str) -> None:
        losses = []
        self._model.train()
        for x, y in tqdm(train_loader):
            x, y = x.to(device).to(torch.float32), y.to(device)
            yhat = self._model(x)
            self._optimizer.zero_grad()
            loss = self._loss(yhat, y.to(torch.long))
            loss.backward()
            losses.append(loss.item())
            self._optimizer.step()
        epsilon = self.privacy_engine.get_epsilon(DELTA)
        print(
            f"Client: {client} "
            f"Train Epoch: {epoch} \t"
            f"Loss: {np.mean(losses):.6f} "
            f" | (ε = {epsilon:.2f})"
        )

    def update_parameters(self, new_parameters: List[torch.Tensor]):
        with torch.no_grad():
            for model_param, new_param in zip(self._model.parameters(), new_parameters):
                model_param.data = new_param.view_as(model_param.data)


In [164]:
def evaluate(model, device, test_loader):
    model.eval()
    loss = torch.nn.CrossEntropyLoss()
    test_loss = 0
    correct   = 0

    predarray  = torch.zeros(len(test_loader.dataset))
    probsarray = torch.zeros((len(test_loader.dataset),2))
    actual     = torch.zeros(len(test_loader.dataset))
    with torch.no_grad():
        for batch_idx, (x, y) in enumerate(tqdm(test_loader)):
            #x, y = x.to(device), y.to(device)
            x, y = x.to(device).to(torch.float32), y.to(device)
            #x, y = x.to(device).to(torch.long), y.to(device).to(torch.long)
            yhat = model(x)

            test_loss    += loss(yhat, y.to(torch.long)).item()
            probabilities = torch.nn.functional.softmax(yhat,1)
            predictions   = torch.argmax(probabilities, 1)
            correct      += predictions.eq(y.view_as(predictions)).sum().item()

            B = 64
            predarray[batch_idx*B:min(len(test_loader.dataset), (batch_idx+1)*B)]  = predictions
            probsarray[batch_idx*B:min(len(test_loader.dataset), (batch_idx+1)*B)] = probabilities
            actual[batch_idx*B:min(len(test_loader.dataset), (batch_idx+1)*B)]     = y

    test_loss /= len(test_loader.dataset)

    print(
        "\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n".format(
            test_loss,
            correct,
            len(test_loader.dataset),
            100.0 * correct / len(test_loader.dataset),
        )
    )
    return correct / len(test_loader.dataset), predarray, probsarray, actual

def plot_confusion_matrix(labels:torch.tensor, predictions:torch.tensor, savefile:str) -> None:
    print("Confusion matrix on test data:")
    print(confusion_matrix(labels, predictions))
    print('Classification Report:')
    print(classification_report(labels, predictions))


In [165]:
import torch

from dataclasses import dataclass

@dataclass
class parameters:
    lr: float              = 0.01
    device: torch.device   = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    sample_rate: float     = 0.2
    #noise_multiplier:float = 1.3
    noise_multiplier:float = 0.5
    max_grad_norm:float    = 1.1
    #DELTA:float     = 1e-7
    DELTA:float     = 1e-4
args = parameters()

In [166]:
import os
import pandas as pd
import numpy as np

# Point directly to your dataset
csv_file = os.path.join("/content/wustl-ehms-2020_with_attacks_categories.csv")

device = args.device
sample_rate = args.sample_rate

# Load dataset
df = pd.read_csv(csv_file)

# --- Clean the dataset properly ---
# Keep only numeric columns + Label
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# Make sure "Label" is included (even if it's categorical in some CSVs)
if "Label" not in numeric_cols and "Label" in df.columns:
    numeric_cols.append("Label")

df = df[numeric_cols].dropna()

print("Columns kept:", df.columns.tolist())
print("Shape after cleaning:", df.shape)

# --- Detect feature size automatically ---
input_features = df.shape[1] - 1   # subtract Label column
print(f"Detected input features: {input_features}")

# Initialize your DataLoader
dataloader = DatasetLoader(df=df, num_clients=2, sample_rate=sample_rate)

# Get train and test loaders
train_loader = dataloader.get_train_data()
test_loader  = dataloader.get_test_data()

# Quick check
for X_batch, y_batch in train_loader:
    print("Train batch X shape:", X_batch.shape)
    print("Train batch y shape:", y_batch.shape)
    break

# Save input_features for model initialization later

Columns kept: ['Dport', 'SrcBytes', 'DstBytes', 'SrcLoad', 'DstLoad', 'SrcGap', 'DstGap', 'SIntPkt', 'DIntPkt', 'SIntPktAct', 'DIntPktAct', 'SrcJitter', 'DstJitter', 'sMaxPktSz', 'dMaxPktSz', 'sMinPktSz', 'dMinPktSz', 'Dur', 'Trans', 'TotPkts', 'TotBytes', 'Load', 'Loss', 'pLoss', 'pSrcLoss', 'pDstLoss', 'Rate', 'Packet_num', 'Temp', 'SpO2', 'Pulse_Rate', 'SYS', 'DIA', 'Heart_rate', 'Resp_Rate', 'ST', 'Label']
Shape after cleaning: (16318, 37)
Detected input features: 36
Train batch X shape: torch.Size([64, 36])
Train batch y shape: torch.Size([64])


In [167]:
class CNN(nn.Module):
    def __init__(self, input_features, num_classes=2):
        super(CNN, self).__init__()

        self.input_size = int(np.sqrt(input_features))  # 6
        assert self.input_size * self.input_size == input_features, \
            "Input features must be a perfect square for CNN reshape"

        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)

        # After conv + pooling → [batch, 32, 1, 1]
        self.fc1 = nn.Linear(32 * 1 * 1, 64)
        self.fc2 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = x.view(-1, 1, self.input_size, self.input_size)  # [batch, 1, 6, 6]
        x = self.pool(F.relu(self.conv1(x)))  # → [batch, 16, 3, 3]
        x = self.pool(F.relu(self.conv2(x)))  # → [batch, 32, 1, 1]
        x = torch.flatten(x, 1)               # → [batch, 32]
        x = F.relu(self.fc1(x))               # → [batch, 64]
        x = self.fc2(x)                       # → [batch, 2]
        return x

In [169]:
# Differential Privacy + CNN (Noise = 0.5)

import time
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score
import pandas as pd

# Timing start
start_time = time.time()

# Initialize clients
clients_data = dataloader.clients_data
clients = {
    client: Client(train_loader,
                   noise_multiplier=0.5,   # DP noise
                   max_grad_norm=1,
                   DELTA=1e-4)
    for client in clients_data
}

# Initialize server with CNN
server = Server(model=CNN(input_features=36, num_classes=2).to(device))

# Federated training
for epoch in range(100):
    for i, (client, client_dataloader) in enumerate(clients_data.items()):
        clients[client].update_parameters(server.parameters)
        clients[client].train_one_epoch(client_dataloader, device, epoch, client)
        server.add_client_parameters(clients[client].parameters, len(client_dataloader.dataset))
    server.aggregate_parameters()

# Timing end (training)
train_time = time.time() - start_time

# Evaluate CNN
test_start = time.time()
accuracy, predictions, probabilities, labels = evaluate(server.model, device, test_loader)
test_time = time.time() - test_start

# ROC Curve
fpr, tpr, thresholds = roc_curve(labels, probabilities[:,1])

# Confusion Matrix Plot
plot_confusion_matrix(labels, predictions, 'federated_privacy_cnn.png')

# Metrics
precision = precision_score(labels, predictions)
recall = recall_score(labels, predictions)
f1 = f1_score(labels, predictions)
roc_auc = roc_auc_score(labels, probabilities[:,1])

print("Final CNN Accuracy:", accuracy)
print("ROC AUC:", roc_auc)
print("\nClassification Report:\n", classification_report(labels, predictions))

# Privacy budget (ε)
# Taking ε from last client as representative (all should be similar)
epsilon = clients[list(clients.keys())[0]].privacy_engine.get_epsilon(1e-4)

# Table Summary
results = pd.DataFrame([{
    "Methods": "Federated + CNN (DP, noise=0.5)",
    "Algorithm": "CNN",
    "Accuracy (%)": round(accuracy * 100, 2),
    "Precision (%)": round(precision * 100, 2),
    "Recall (%)": round(recall * 100, 2),
    "F1-Score (%)": round(f1 * 100, 2),
    "Train Time (s)": round(train_time, 2),
    "Test Time (s)": round(test_time, 2),
    "Privacy Budget (ε)": round(epsilon, 2)
}])

print("\n=== Results Table ===")
print(results.to_string(index=False))


  0%|          | 0/26 [00:00<?, ?it/s]/tmp/ipython-input-1682663543.py:88: UserWarning: Full backward hook is firing when gradients are computed with respect to module outputs since no inputs require gradients. See https://docs.pytorch.org/docs/main/generated/torch.nn.Module.html#torch.nn.Module.register_full_backward_hook for more details.
  loss.backward()
100%|██████████| 26/26 [00:00<00:00, 51.85it/s]


Client: client_0 Train Epoch: 0 	Loss: 0.690524  | (ε = 5.17)


100%|██████████| 26/26 [00:00<00:00, 52.42it/s]


Client: client_1 Train Epoch: 0 	Loss: 0.690741  | (ε = 5.17)


100%|██████████| 26/26 [00:00<00:00, 53.06it/s]


...

Client: client_0 Train Epoch: 99 	Loss: 0.861249  | (ε = 34.78)


100%|██████████| 26/26 [00:00<00:00, 47.98it/s]


Client: client_1 Train Epoch: 99 	Loss: 0.897972  | (ε = 34.78)


100%|██████████| 52/52 [00:00<00:00, 150.96it/s]



Test set: Average loss: 0.0075, Accuracy: 2745/3279 (83.71%)

Confusion matrix on test data:
[[2564  316]
 [ 218  181]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.92      0.89      0.91      2880
         1.0       0.36      0.45      0.40       399

    accuracy                           0.84      3279
   macro avg       0.64      0.67      0.65      3279
weighted avg       0.85      0.84      0.84      3279

Final CNN Accuracy: 0.8371454711802379
ROC AUC: 0.772389306599833

Classification Report:
               precision    recall  f1-score   support

         0.0       0.92      0.89      0.91      2880
         1.0       0.36      0.45      0.40       399

    accuracy                           0.84      3279
   macro avg       0.64      0.67      0.65      3279
weighted avg       0.85      0.84      0.84      3279


=== Results Table ===
                        Methods Algorithm  Accuracy (%)  Precision (%)  Recall (%)  F1-S